Este script carga varios conjuntos de datos, los limpia y los integra para realizar:
Resumen diario y mensual: se procesan los datos climáticos y se generan un resumen diario y luego uno mensual. Este resumen incluye el cálculo de los días consecutivos que superan ciertos umbrales de temperatura y humedad.
Integración de datos: se combinan los datos climáticos, de mortalidad y de olas de calor, y se normalizan los nombres de las provincias para asegurar la consistencia entre los archivos.
Filtrado: Se filtran los registros de años y meses relevantes (excluyendo 2020, 2023 y 2024, y seleccionando solo los meses de mayo a septiembre).
Visualización: Se generan visualizaciones, como boxplots y un histograma, para analizar la mortalidad y las variables climáticas por provincia.
Estadísticas descriptivas: se generan estadísticas descriptivas del conjunto de datos filtrado.

In [ ]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from fuzzywuzzy import process

# Configuración de gráficos
sns.set(style="whitegrid")

# 1. Definir las rutas de archivos de entrada y salida
data_dir = os.path.join('data')
result_dir = os.path.join(data_dir, 'result')
if not os.path.exists(result_dir):
    os.makedirs(result_dir)

file_paths = {
    'clima': os.path.join(data_dir, 'archivo_clima.csv'),
    'mortalidad': os.path.join(data_dir, 'mortalidad_completado.csv'),
    'olas_calor': os.path.join(data_dir, 'olas_de_calor_2.csv')
}

# 2. Función para cargar archivos con diferentes codificaciones
def load_data(path):
    for encoding in ['utf-8', 'ISO-8859-1', 'windows-1252']:
        try:
            return pd.read_csv(path, encoding=encoding, low_memory=False)
        except UnicodeDecodeError:
            continue
    raise ValueError(f"No se pudo leer el archivo {path} con las codificaciones comunes")

# Cargar los datos necesarios
df_clima = load_data(file_paths['clima'])
df_mortalidad = load_data(file_paths['mortalidad'])
df_olas_calor = load_data(file_paths['olas_calor'])

# 3. Procesar los datos climáticos (resumen diario)
# Seleccionar las columnas necesarias
columns_to_keep = ['provincia', 'altitud', 'tmin', 'tmax', 'prec', 'fecha', 'hrMedia']
df_clima_clean = df_clima[columns_to_keep]
df_clima_clean.dropna(how='all', inplace=True)
df_clima_clean['fecha'] = pd.to_datetime(df_clima_clean['fecha'], format='%Y-%m-%d', errors='coerce')

# Resumen diario
resumen_diario = df_clima_clean.groupby(['provincia', 'fecha']).agg(
    temp_min=('tmin', 'min'),
    temp_max=('tmax', 'max'),
    humedad_max=('hrMedia', 'max'),
    prec_max=('prec', 'max'),
    altitud_prom=('altitud', 'mean')
).reset_index()

# Guardar el resumen diario en CSV
daily_summary_path = os.path.join(result_dir, 'resumen_diario_clima.csv')
resumen_diario.to_csv(daily_summary_path, index=False)
print(f"Resumen diario guardado en: {daily_summary_path}")

# 4. Generar el resumen mensual
resumen_diario['mes'] = resumen_diario['fecha'].dt.to_period('M')

# Función para contar días consecutivos que superan umbrales
def contar_dias_consecutivos(serie, umbral):
    return serie.gt(umbral).astype(int).groupby((serie.gt(umbral) != serie.gt(umbral).shift()).cumsum()).cumsum().max()

# Umbrales de temperatura y humedad
umbrales_tmin = [22, 25, 28]
umbrales_tmax = [35, 38, 41]
umbrales_humedad = [70, 80, 90]

# Contar días consecutivos por umbral
for umbral in umbrales_tmin:
    resumen_diario[f'dias_consecutivos_tmin_{umbral}'] = resumen_diario.groupby(['provincia', 'mes'])['temp_min'].transform(lambda x: contar_dias_consecutivos(x, umbral))
for umbral in umbrales_tmax:
    resumen_diario[f'dias_consecutivos_tmax_{umbral}'] = resumen_diario.groupby(['provincia', 'mes'])['temp_max'].transform(lambda x: contar_dias_consecutivos(x, umbral))
for umbral in umbrales_humedad:
    resumen_diario[f'dias_consecutivos_humedad_{umbral}'] = resumen_diario.groupby(['provincia', 'mes'])['humedad_max'].transform(lambda x: contar_dias_consecutivos(x, umbral))

# Crear resumen mensual
resumen_mensual = resumen_diario.groupby(['provincia', 'mes']).agg(
    temp_min_max=('temp_min', 'max'),
    temp_max_max=('temp_max', 'max'),
    humedad_max=('humedad_max', 'max'),
    prec_sum=('prec_max', 'sum'),
    altitud_prom=('altitud_prom', 'mean')
).reset_index()

# Guardar el resumen mensual en CSV
monthly_summary_path = os.path.join(result_dir, 'resumen_mensual_clima.csv')
resumen_mensual.to_csv(monthly_summary_path, index=False)
print(f"Resumen mensual guardado en: {monthly_summary_path}")

# 5. Integrar datos de mortalidad y olas de calor
df_mortalidad = df_mortalidad.melt(id_vars=['Provincias'], var_name='mes', value_name='mortalidad')
df_mortalidad['mes'] = df_mortalidad['mes'].str.replace('M', '-')
df_mortalidad['mes'] = pd.to_datetime(df_mortalidad['mes'], format='%Y-%m').dt.to_period('M').astype(str)
df_mortalidad.rename(columns={'Provincias': 'provincia'}, inplace=True)

df_olas_calor['provincias'] = df_olas_calor['provincias'].str.replace('"', '').str.split(',')
olas_calor_expandido = df_olas_calor.explode('provincias').reset_index(drop=True)
olas_calor_expandido.rename(columns={'provincias': 'provincia'}, inplace=True)

# Combinar resumen mensual con mortalidad y olas de calor
df_combined = pd.merge(resumen_mensual, df_mortalidad, on=['provincia', 'mes'], how='left')
df_combined = pd.merge(df_combined, olas_calor_expandido[['provincia', 'mes', 'dias']], on=['provincia', 'mes'], how='left')
df_combined['dias'].fillna(0, inplace=True)

# Guardar el archivo combinado filtrado
combined_output_path = os.path.join(result_dir, 'dataset_combined_filtrado.csv')
df_combined.to_csv(combined_output_path, index=False)
print(f"Archivo combinado guardado en: {combined_output_path}")

# ------------------------------------------------------
# Segundo bloque: Filtrado y visualización
# ------------------------------------------------------

# 6. Cargar el dataset combinado
df_combined = pd.read_csv(combined_output_path)

# 7. Limpiar y normalizar los datos
df_combined['provincia'] = df_combined['provincia'].str.strip().str.upper()

# Reemplazar nombres de provincias incorrectos
reemplazos_provincias = {
    'A CORUA': 'A CORUÑA', 'CADIZ': 'CÁDIZ', 'CORDOBA': 'CÓRDOBA',
    'CACERES': 'CÁCERES', 'CASTELLON': 'CASTELLÓ', 'VALENCIA': 'VALÈNCIA',
    'ALICANTE': 'ALACANT', 'LEON': 'LEÓN', 'ALMERIA': 'ALMERÍA', 'MELILLA': 'MELILLA',
    'STA CRUZ DE TENERIFE': 'SANTA CRUZ DE TENERIFE', 'BADAJOZ': 'BADAJOZ',
    'JAEN': 'JAÉN', 'CEUTA': 'CEUTA', 'AVILA': 'ÁVILA', 'MALAGA': 'MÁLAGA',
    'ARABAALAVA': 'ARABA', 'SEVILLA': 'SEVILLA',
    'TERRITORIO NO ASOCIADO A NINGUNA PROVINCIA': 'TERRITORIO NO ASOCIADO A NINGUNA PROVINCIA'
}
df_combined['provincia'] = df_combined['provincia'].replace(reemplazos_provincias)

# 8. Filtrar por fechas y columnas relevantes
df_combined['fecha'] = pd.to_datetime(df_combined['mes'], format='%Y-%m')
df_combined['año'] = df_combined['fecha'].dt.year
df_combined['mes_num'] = df_combined['fecha'].dt.month

# Filtrar años y meses
df_filtered = df_combined[~df_combined['año'].isin([2020, 2023, 2024])]
df_filtered = df_filtered[(df_filtered['mes_num'] >= 5) & (df_filtered['mes_num'] <= 9)]

# Guardar el archivo filtrado
filtered_output_path = os.path.join(result_dir, 'dataset_preparado.csv')
df_filtered.to_csv(filtered_output_path, index=False)
print(f"Archivo filtrado guardado en: {filtered_output_path}")

# 9. Visualización de los datos filtrados
# Distribución de temperatura máxima por provincia
plt.figure(figsize=(12, 6))
sns.boxplot(x='provincia', y='temp_max_max', data=df_filtered)
plt.xticks(rotation=90)
plt.title("Distribución de la Temperatura Máxima por Provincia")
plt.xlabel("Provincia")
plt.ylabel("Temperatura Máxima (°C)")
plt.grid(True)
plt.savefig(os.path.join(result_dir, 'boxplot_temp_max_provincia.png'))
plt.show()

# Distribución de la mortalidad por provincia
plt.figure(figsize=(12, 6))
sns.boxplot(x='provincia', y='mortalidad', data=df_filtered)
plt.xticks(rotation=90)
plt.title("Distribución de la Mortalidad por Provincia")
plt.xlabel("Provincia")
plt.ylabel("Mortalidad")
plt.grid(True)
plt.savefig(os.path.join(result_dir, 'boxplot_mortalidad_provincia.png'))
plt.show()

# Histograma de la mortalidad
plt.figure(figsize=(10, 6))
sns.histplot(df_filtered['mortalidad'], bins=30, kde=True, color='blue')
plt.title("Distribución de la Mortalidad")
plt.xlabel("Mortalidad")
plt.ylabel("Frecuencia")
plt.grid(True)
plt.savefig(os.path.join(result_dir, 'histograma_mortalidad.png'))
plt.show()

# 10. Generar estadísticas descriptivas del dataset filtrado
estadisticas_descriptivas = df_filtered.describe()

# Guardar estadísticas en CSV
estadisticas_output_path = os.path.join(result_dir, 'estadisticas_descriptivas.csv')
estadisticas_descriptivas.to_csv(estadisticas_output_path)
print(f"Estadísticas descriptivas guardadas en: {estadisticas_output_path}")
